# Banana Bot
## Samuel Horovatin, 11185403, sch923

In [9]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os, datetime
import math
import matplotlib.pyplot as plt

# General Network Constants
EPOCHS = 10
BATCH = 32
OPTIMIZER = tf.keras.optimizers.Adam(1e-3)

# Data Imports Constants 
DATA_PATH = 'G:\\shoro\\OneDrive - Personal\\OneDrive\School\\2020-2021\\CMPT898\\CMPT898_Repo\\Project\\data\\bananas\\'
LABEL_PATH = "Estu.csv"
VALIDATION_SPLIT = 0.5
SEED = 1996
TARGET_SIZE = (224,224)
HEIGHT_SHIFT = 0.10
WIDTH_SHIFT = [-22,22]
ZOOM_RANGE  = 0.10 
K_FOLDS = 10

# Network Construction Constants
INPUT_SHAPE = (224,224,3)
CONV_DEPTH = 256 # Taken from https://github.com/potterhsu/easy-fpn.pytorch. Not 100% sure why it is 256...

def MRD(y_true, y_pred):
    # y_pred = y_pred[:, 0]
    return tf.keras.backend.sum(tf.keras.backend.abs(y_pred - y_true) / y_true )

def One_FVU(y_true, y_pred):
    # y_pred = y_pred[:, 0]
    y_mean = tf.keras.backend.mean(y_true)
    num = tf.math.squared_difference(y_true, y_pred)
    den = tf.keras.backend.sum(y_true - y_mean)
    return 1 - ( num / den )

# As described by Kendall and Gal: https://proceedings.neurips.cc/paper/2017/hash/2650d6089a6d640c5e85b2b88265dc2b-Abstract.html
# For regression, output_D should always be 1
# def Uncertainty_Loss(y_true, y_pred):
#     mu = y_pred[:, 0]
#     si = y_pred[:, 1]
#     loss = (si + tf.math.squared_difference(y_true, mu) / tf.math.exp(si)) / 2.0
#     return tf.reduce_mean(loss)


METRICS = [MRD, One_FVU]
# LOSS = Uncertainty_Loss

In [2]:
# Gets labels from label csv
data_label_df = pd.read_csv(LABEL_PATH)

# Create training and testing datasets
dat_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255, 
    validation_split=VALIDATION_SPLIT,
    height_shift_range=HEIGHT_SHIFT,
    width_shift_range=WIDTH_SHIFT,
    zoom_range=ZOOM_RANGE,
    fill_mode="wrap"
)

train_datagen = dat_gen.flow_from_dataframe(
    dataframe=data_label_df,
    directory=DATA_PATH,
    x_col='File Name',
    y_col='Weight',
    target_size=TARGET_SIZE,
    batch_size=BATCH,
    class_mode='raw',
    shuffle=True,
    seed=SEED,
    subset='training'
)

valid_datagen = dat_gen.flow_from_dataframe(
    dataframe=data_label_df,
    directory=DATA_PATH,
    x_col='File Name',
    y_col='Weight',
    target_size=TARGET_SIZE,
    batch_size=BATCH,
    class_mode='raw',
    shuffle=True,
    seed=SEED,
    subset="validation"
)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

test_datagen = test_datagen.flow_from_dataframe(
    dataframe=data_label_df,
    directory=DATA_PATH,
    x_col='File Name',
    y_col='Weight',
    target_size=TARGET_SIZE,
    batch_size=BATCH,
    class_mode='raw',
    shuffle=True,
    seed=SEED
)

Found 285 validated image filenames.
Found 284 validated image filenames.
Found 569 validated image filenames.


In [3]:
# x_batch, y_batch = next(train_datagen)

# # prints out a sample of the regularized images
# for i in range (0,16):
#     image = x_batch[i]
#     plt.imshow(image)
#     plt.show()

In [10]:
# Trains the model given the constant parameters and returns the model and the fit history
def Train_Model(model, train_datagen, test_datagen, valid_datagen, loss_func):
  model.compile(optimizer=OPTIMIZER,
                loss=loss_func,
                metrics=METRICS,
                run_eagerly=True # For debugging
                )

  csv_logger = tf.keras.callbacks.CSVLogger('training.log', separator=',')
  # early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.03, mode='min', patience=8)

  STEP_SIZE_TRAIN=train_datagen.n//train_datagen.batch_size
  STEP_SIZE_VALID=valid_datagen.n//valid_datagen.batch_size
  STEP_SIZE_TEST=test_datagen.n//test_datagen.batch_size

  history = model.fit(x=train_datagen,
                      steps_per_epoch=STEP_SIZE_TRAIN,
                      validation_data=valid_datagen,
                      validation_steps=STEP_SIZE_VALID, 
                      epochs=EPOCHS,
                      callbacks=[csv_logger])
        
  return (model, history)

In [11]:
# Takes in an model and returns a list of edges last layers name in a block
# Layers within blocks need to be named with '*block#*' were # is the block number
def blockEdgeNameFinder(model):
    layer_index = 0 # This allows the function to skip the start of the network
    current_block = "block"
    block_edges = []
    last_layer_name = ""

    for layer in model.layers:
        if current_block in layer.name:
            if layer_index == 0:
                layer_index = layer_index + 1
                current_block = f"block{layer_index}"
                
        elif layer_index == 0:
            continue

        elif "block" in last_layer_name:
            layer_index = layer_index + 1
            next_block = f"block{layer_index}"
            current_block = next_block
            block_edges.append(last_layer_name)
        last_layer_name = layer.name

    return block_edges

# Takes in an model (which is a layer) and a name for the topper and adds the regression topper as described by the paper
def Add_Regression_Top(model, name):
    options = {
        'activation'         : 'relu',
        'kernel_initializer' : tf.keras.initializers.HeNormal()
    }
    outputs = tf.keras.layers.Conv2D(
        256,
        model.shape[1:3],
        activation = options['activation'],
        kernel_initializer= options['kernel_initializer'], 
        name=f'{name}_conv1_256'
    )(model)
    outputs = tf.keras.layers.Conv2D(
        256,
        outputs.shape[1:3],
        activation = options['activation'],
        kernel_initializer= options['kernel_initializer'], 
        name=f'{name}_conv2_256'
    )(outputs)

    outputs = tf.keras.layers.GlobalAveragePooling2D()(outputs)
    outputs = tf.keras.layers.Dense(
        256,
        activation = options['activation'],
        kernel_initializer= options['kernel_initializer'], 
        name=f'{name}_dense_256'
    )(outputs)

    outputs = tf.keras.layers.Dense(
        128,
        activation = options['activation'],
        kernel_initializer= options['kernel_initializer'], 
        name=f'{name}_dense_128'
    )(outputs)

    outputs = tf.keras.layers.Dense(
        64,
        activation = options['activation'],
        kernel_initializer= options['kernel_initializer'], 
        name=f'{name}_dense_64'
    )(outputs)
    # outputs = tf.keras.layers.Dense(
    #     2, 
    #     kernel_initializer= options['kernel_initializer'],
    #     name=f'{name}_out'
    # )(outputs) 
    outputs = tf.keras.layers.Dense(
        1, 
        kernel_initializer= options['kernel_initializer'],
        name=f'{name}_out'
    )(outputs)
    
    return outputs

# Final consolidation layer of FPN predictions
class WeightedAverage(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(WeightedAverage, self).__init__(**kwargs)

    def call(self, model_outputs):
        model_outputs = tf.convert_to_tensor(model_outputs)
        estimates = model_outputs[:, 0]
        variances = model_outputs[:, 1]
        
        weighted_avg = tf.cast(tf.math.reduce_sum(tf.math.divide(estimates, variances)),'float32')

        # based on the wikipedia page for weighted variance average
        avg_variance = tf.cast(tf.math.reduce_mean(variances),'float32')
        return avg_variance * weighted_avg

In [6]:

def EfficentNet(B_lvl):
    basemodel = tf.keras.applications.EfficientNetB0(input_shape=INPUT_SHAPE, include_top=False, weights='imagenet')

    if B_lvl == 'B0' or B_lvl == 0:
        return build_EfficentNet_B0(basemodel)
    elif B_lvl == 'B1' or B_lvl == 1:
        print(f"EfficentNet Error: Level {B_lvl} unimplemented.")
    elif B_lvl == 'B2' or B_lvl == 2:
        print(f"EfficentNet Error: Level {B_lvl} unimplemented.")
    elif B_lvl == 'B3' or B_lvl == 3:
        print(f"EfficentNet Error: Level {B_lvl} unimplemented.")
    elif B_lvl == 'B4' or B_lvl == 4:
        print(f"EfficentNet Error: Level {B_lvl} unimplemented.")
    elif B_lvl == 'B5' or B_lvl == 5:
        print(f"EfficentNet Error: Level {B_lvl} unimplemented.")
    elif B_lvl == 'B6' or B_lvl == 6:
        print(f"EfficentNet Error: Level {B_lvl} unimplemented.")
    elif B_lvl == 'B7' or B_lvl == 7:
        print(f"EfficentNet Error: Level {B_lvl} unimplemented")
    else:
        print(f"EfficentNet Error: Unknown efficentnet level {B_lvl}")

def build_EfficentNet_B0(basemodel):
    FPN_LAYER_NAMES = blockEdgeNameFinder(basemodel)
    backbone = []

    FPN_LAYER_NAMES = FPN_LAYER_NAMES[2:]
    FPN_LAYER_NAMES = FPN_LAYER_NAMES[::-1] 

    for name in FPN_LAYER_NAMES: 
        backbone.append(basemodel.get_layer(name).output)

    prediction_outputs = [] 
    # variance_outputs = [] 
    first_layer = True
    layer_index = 7
    for layer in backbone: 
        # FPN consolidation
        FPN_layer = tf.keras.layers.Conv2D(filters=CONV_DEPTH, kernel_size=(1,1), name=f"fpn{layer_index}_conv")(layer) 
        if not first_layer:
            A = last_FPN_layer.shape[1:][1]
            B = FPN_layer.shape[1:][1] # All layers should be square resolution
            scale = int(round(B/A))
            last_FPN_layer = tf.keras.layers.UpSampling2D((scale,scale), name=f"fpn{layer_index}_scale{scale}")(last_FPN_layer)
            FPN_layer = tf.keras.layers.Add(name=f"fpn{layer_index}_add")([FPN_layer, last_FPN_layer])
            
        else:
            first_layer = False
        last_FPN_layer = FPN_layer
        new_prediction = Add_Regression_Top(FPN_layer, f"fpn_reg{layer_index}")
        prediction_outputs.append(new_prediction)
        # variance_outputs.append(new_variance)
        layer_index = layer_index - 1
    print(f"Model Outputs{prediction_outputs}")
    final_model = tf.keras.models.Model(inputs=basemodel.inputs, outputs=prediction_outputs, name="EfficentNet_FPN")
    return final_model

test_model = tf.keras.applications.EfficientNetB0(input_shape=INPUT_SHAPE, include_top=False, weights='imagenet')    
rebuilt_model = build_EfficentNet_B0(test_model)
rebuilt_model.summary()

Model Outputs[<tf.Tensor 'fpn_reg7_out/BiasAdd:0' shape=(None, 1) dtype=float32>, <tf.Tensor 'fpn_reg6_out/BiasAdd:0' shape=(None, 1) dtype=float32>, <tf.Tensor 'fpn_reg5_out/BiasAdd:0' shape=(None, 1) dtype=float32>, <tf.Tensor 'fpn_reg4_out/BiasAdd:0' shape=(None, 1) dtype=float32>, <tf.Tensor 'fpn_reg3_out/BiasAdd:0' shape=(None, 1) dtype=float32>]
Model: "EfficentNet_FPN"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 224, 224, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalizatio

In [12]:
def Uncertainty_Loss(y_true, y_pred):
    mu = y_pred[:, 0]
    si = y_pred[:, 1]
    loss = (si + tf.math.squared_difference(y_true, mu) / tf.math.exp(si)) / 2.0
    return tf.reduce_mean(loss)

Train_Model(rebuilt_model, train_datagen, test_datagen, valid_datagen, tf.keras.losses.MSE)

Epoch 1/10
8/8 [==============================] - 9s 1s/step - loss: 9060.3604 - fpn_reg7_out_loss: 2.7029 - fpn_reg6_out_loss: 19.8953 - fpn_reg5_out_loss: 159.8554 - fpn_reg4_out_loss: 340.6282 - fpn_reg3_out_loss: 8537.2773 - fpn_reg7_out_MRD: 5.9197 - fpn_reg7_out_One_FVU: -596501.0625 - fpn_reg6_out_MRD: 14.1337 - fpn_reg6_out_One_FVU: -26445456.0000 - fpn_reg5_out_MRD: 38.3014 - fpn_reg5_out_One_FVU: 180715600.0000 - fpn_reg4_out_MRD: 61.3482 - fpn_reg4_out_One_FVU: -412637216.0000 - fpn_reg3_out_MRD: 299.9151 - fpn_reg3_out_One_FVU: -8502816256.0000 - val_loss: 92636258304.0000 - val_fpn_reg7_out_loss: 850848640.0000 - val_fpn_reg6_out_loss: 150566640.0000 - val_fpn_reg5_out_loss: 1917442560.0000 - val_fpn_reg4_out_loss: 24678721536.0000 - val_fpn_reg3_out_loss: 65038680064.0000 - val_fpn_reg7_out_MRD: 127595.8281 - val_fpn_reg7_out_One_FVU: -inf - val_fpn_reg6_out_MRD: 47285.2148 - val_fpn_reg6_out_One_FVU: -inf - val_fpn_reg5_out_MRD: 195695.0938 - val_fpn_reg5_out_One_FVU: -i

TypeError: object of type 'NoneType' has no len()

In [8]:
network_output = [
    tf.constant([81, 2]),
    tf.constant([74, 7]),
    tf.constant([20, 19]),
    tf.constant([11, 22]),
    tf.constant([2, 26])
]

class WeightedAverage(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(WeightedAverage, self).__init__(**kwargs)

    def call(self, model_outputs):
        model_outputs = tf.convert_to_tensor(model_outputs)
        estimates = model_outputs[:, 0]
        variances = model_outputs[:, 1]
        
        weighted_avg = tf.cast(tf.math.reduce_sum(tf.math.divide(estimates, variances)),'float32')

        # based on the wikipedia page for weighted variance average
        avg_variance = tf.cast(tf.math.reduce_mean(variances),'float32')
        print(f"estimates {estimates}, variances {variances}, weighted_avg {weighted_avg}, avg_variance {avg_variance}")
        return avg_variance * weighted_avg
print(WeightedAverage()(network_output))

estimates [81 74 20 11  2], variances [ 2  7 19 22 26], weighted_avg 52.700984954833984, avg_variance 15.0
tf.Tensor(790.5148, shape=(), dtype=float32)
